In [22]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re

In [23]:
html = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup=BeautifulSoup(html.content,features='html.parser')
table=soup.find('table')
rows=table.findAll('tr')
table_data={}
ix=0
for r in rows:
    row_data=r.findAll('td')
    if row_data:
        #assume that there are only 3 columns
        table_data.update({
            ix:[
                row_data[0].text.replace("\n",""), 
                row_data[1].text.replace("\n",""), 
                row_data[2].text.replace("/", ",").replace("\n", ""),
            ]
        })
        ix+=1

In [24]:
columns=['Postal Code', 'Borough', 'Neighborhoods']

In [25]:
df = pd.DataFrame.from_dict(data=table_data, orient='index', columns=columns)

In [26]:
df.head()

,Postal Code,Borough,Neighborhoods
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park , Harbourfront"


In [27]:
new_data={}
for index, row in df.iterrows():
    if row['Borough']=='Not assigned':
        continue
    
    if row['Neighborhoods']=='Not assigned':
        row['Neighborhoods']=row['Borough']
        
    if row['Postal Code'] in new_data.keys():
        if row['Neighborhoods'] not in new_data[row['Postal Code']][2]:
            row['Neighborhoods'] = new_data[row['Postal Code']][2] + " , " + row['Neighborhoods']
        
    new_data.update({
        row['Postal Code']: [row[0],row[1],row[2]]
    })
    
# steps to create dictionary for the DataFrame
new_data_new={}
xcount=0
for i in new_data.keys():
    new_data_new.update({
        xcount: new_data[i]
    })
    xcount+=1
new_df=pd.DataFrame.from_dict(data=new_data_new, orient='index', columns=columns)

In [28]:
new_df.head()

,Postal Code,Borough,Neighborhoods
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"


In [29]:
new_df.shape

(103, 3)